<a href="https://colab.research.google.com/github/tahmidawal/AWS_Lambda_metadata_update_automation/blob/main/DataBaseCreation%2BHeatMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sqlite3


In [20]:
# Adjust the file paths as necessary
healthcare_orgs_df = pd.read_csv('/content/healthcare_orgs_usa (2).csv')
population_zip_df = pd.read_csv('/content/DECENNIALDHC2020.P1-Data.csv')


In [21]:
# Assuming the correct ZIP code is in the 'NAME' column, formatted like "ZCTA5 12345"
population_zip_df['Zip_Code'] = population_zip_df['NAME'].str.extract('(\d{5})')
population_zip_df = population_zip_df[['Zip_Code', 'P1_001N']].rename(columns={"P1_001N": "Population"})

# Re-insert the corrected data into the database
population_zip_df.to_sql('population_zip', conn, if_exists='replace', index=False)


33775

In [22]:
# Create or connect to a SQLite database
conn = sqlite3.connect('healthcare_population.db')

# Create tables in the database
conn.execute("""
    CREATE TABLE IF NOT EXISTS healthcare_orgs (
        City TEXT,
        Location TEXT,
        Name TEXT,
        Address TEXT,
        Zip_Code TEXT,
        Place_ID TEXT,
        Latitude REAL,
        Longitude REAL,
        Rating REAL,
        User_Ratings_Total INTEGER,
        Website TEXT,
        Phone_Number TEXT,
        Description TEXT
    )
""")

conn.execute("""
    CREATE TABLE IF NOT EXISTS population_zip (
        Zip_Code TEXT,
        Population INTEGER
    )
""")


In [23]:
# Insert data into the tables
healthcare_orgs_df.to_sql('healthcare_orgs', conn, if_exists='replace', index=False)
population_zip_df.to_sql('population_zip', conn, if_exists='replace', index=False)


33775

In [24]:
# Query to check sample data from healthcare_orgs
print(pd.read_sql("SELECT * FROM healthcare_orgs LIMIT 5", conn))

# Query to check sample data from population_zip
print(pd.read_sql("SELECT * FROM population_zip LIMIT 5", conn))


           City                   Location  \
0  Aberdeen, NC  hospitals in Aberdeen, NC   
1  Aberdeen, NC  hospitals in Aberdeen, NC   
2  Aberdeen, NC  hospitals in Aberdeen, NC   
3  Aberdeen, NC  hospitals in Aberdeen, NC   
4  Aberdeen, NC  hospitals in Aberdeen, NC   

                                             Name  \
0             FirstHealth Moore Regional Hospital   
1               PORT Health | Aberdeen Adolescent   
2                             FastMed Urgent Care   
3  FirstHealth Moore Regional Hospital - Richmond   
4   Pinehurst Medical Clinic Primary Care - South   

                                             Address  Zip Code  \
0  155 Memorial Dr, Pinehurst, NC 28374, United S...   28374.0   
1   206 N Pine St, Aberdeen, NC 28315, United States   28315.0   
2  1800 N Sandhills Blvd, Aberdeen, NC 28315, Uni...   28315.0   
3  925 S Long Dr, Rockingham, NC 28379, United St...   28379.0   
4  1902 N Sandhills Blvd, Aberdeen, NC 28315, Uni...   28315.0   

       

In [26]:
# Corrected SQL query to calculate the number of clinics per ZIP code
query = """
    SELECT h.[Zip Code] AS Zip_Code, COUNT(*) AS Clinics_Count, p.Population
    FROM healthcare_orgs h
    JOIN population_zip p ON h.[Zip Code] = p.Zip_Code
    GROUP BY h.[Zip Code]
"""

clinics_per_zip_df = pd.read_sql(query, conn)



In [28]:
clinics_per_zip_df.to_sql('clinics_per_zip_df', conn, if_exists='replace', index=False)


700

In [29]:
conn.close()


In [34]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('healthcare_population.db')

# Load the data from the database
healthcare_data = pd.read_sql("SELECT * FROM healthcare_orgs", conn)
clinics_per_zip_df = pd.read_sql("SELECT * FROM clinics_per_zip_df", conn)

# Calculate average latitude and longitude for each ZIP code
# Use the correct column name here
avg_lat_lon = healthcare_data.groupby('Zip Code').agg({'Latitude': 'mean', 'Longitude': 'mean'}).reset_index()

# Rename the column to match the column in clinics_per_zip_df for merging
avg_lat_lon = avg_lat_lon.rename(columns={'Zip Code': 'Zip_Code'})

# Merge this with the clinics_per_zip_df
clinics_per_zip_df = clinics_per_zip_df.merge(avg_lat_lon, on='Zip_Code', how='left')

# Create a map object, starting at a central location
mymap = folium.Map(location=[37.0902, -95.7129], zoom_start=4)  # USA central coordinates

# Prepare data for the heatmap
heatmap_data = clinics_per_zip_df[['Latitude', 'Longitude', 'Clinics_Count']].values.tolist()

# Add the heatmap layer
HeatMap(heatmap_data).add_to(mymap)

# Save or display the map
mymap.save('healthcare_orgs_with_heatmap.html')  # Change the path as needed

# Close the database connection
conn.close()


In [36]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('healthcare_population.db')

# Load the data from the database
healthcare_data = pd.read_sql("SELECT * FROM healthcare_orgs", conn)
clinics_per_zip_df = pd.read_sql("SELECT * FROM clinics_per_zip_df", conn)

# Calculate average latitude and longitude for each ZIP code
avg_lat_lon = healthcare_data.groupby('Zip Code').agg({'Latitude': 'mean', 'Longitude': 'mean'}).reset_index()

# Rename the column to match the column in clinics_per_zip_df for merging
avg_lat_lon = avg_lat_lon.rename(columns={'Zip Code': 'Zip_Code'})

# Merge this with the clinics_per_zip_df
clinics_per_zip_df = clinics_per_zip_df.merge(avg_lat_lon, on='Zip_Code', how='left')

# Create a map object, starting at a central location
mymap = folium.Map(location=[37.0902, -95.7129], zoom_start=4)  # USA central coordinates

# Loop through the healthcare data and add markers for each clinic
for idx, row in healthcare_data.iterrows():
    popup_text = f"{row['Name']}<br>Rating: {row['Rating']}<br>{row['Address']}"
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup_text,
        icon=folium.Icon(icon="plus", color="blue")
    ).add_to(mymap)

# Loop through the clinics_per_zip_df and add popups for the number of clinics in each ZIP code
for idx, row in clinics_per_zip_df.iterrows():
    popup_text = f"ZIP Code: {row['Zip_Code']}<br>Clinics Count: {row['Clinics_Count']}"
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup_text,
        icon=folium.Icon(icon="info-sign", color="red")
    ).add_to(mymap)

# Prepare data for the heatmap
heatmap_data = clinics_per_zip_df[['Latitude', 'Longitude', 'Clinics_Count']].values.tolist()

# Add the heatmap layer
HeatMap(heatmap_data).add_to(mymap)

# Save or display the map
mymap.save('healthcare_orgs_with_heatmap.html')  # Change the path as needed

# Close the database connection
conn.close()


In [37]:
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('healthcare_population.db')

# Load the data from the database
healthcare_data = pd.read_sql("SELECT * FROM healthcare_orgs", conn)
clinics_per_zip_df = pd.read_sql("SELECT * FROM clinics_per_zip_df", conn)

# Calculate average latitude and longitude for each ZIP code
avg_lat_lon = healthcare_data.groupby('Zip Code').agg({'Latitude': 'mean', 'Longitude': 'mean'}).reset_index()

# Rename the column to match the column in clinics_per_zip_df for merging
avg_lat_lon = avg_lat_lon.rename(columns={'Zip Code': 'Zip_Code'})

# Merge this with the clinics_per_zip_df
clinics_per_zip_df = clinics_per_zip_df.merge(avg_lat_lon, on='Zip_Code', how='left')

# Create a map object, starting at a central location
mymap = folium.Map(location=[37.0902, -95.7129], zoom_start=4)  # USA central coordinates

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(mymap)

# Loop through the healthcare data and add markers to the cluster
for idx, row in healthcare_data.iterrows():
    popup_text = f"{row['Name']}<br>Rating: {row['Rating']}<br>{row['Address']}"
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup_text
    ).add_to(marker_cluster)

# Loop through the clinics_per_zip_df and add popups for the number of clinics in each ZIP code
for idx, row in clinics_per_zip_df.iterrows():
    popup_text = f"ZIP Code: {row['Zip_Code']}<br>Clinics Count: {row['Clinics_Count']}"
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup_text,
        icon=folium.Icon(icon="info-sign", color="red")
    ).add_to(marker_cluster)

# Prepare data for the heatmap
heatmap_data = clinics_per_zip_df[['Latitude', 'Longitude', 'Clinics_Count']].values.tolist()

# Add the heatmap layer
HeatMap(heatmap_data).add_to(mymap)

# Save or display the map
mymap.save('healthcare_orgs_with_heatmap.html')  # Change the path as needed

# Close the database connection
conn.close()
